Working code

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.applications.efficientnet import preprocess_input
import os

# Preprocess image for EfficientNet
def preprocess_for_effnet(crop):
    crop_resized = cv2.resize(crop, (300, 300))  # Resize to 300x300 for EfficientNet B3
    crop_preprocessed = preprocess_input(crop_resized)  # Apply EfficientNet preprocessing
    crop_preprocessed = np.expand_dims(crop_preprocessed, axis=0)  # Add batch dimension
    return crop_preprocessed

# Detection and classification pipeline for a single image
def detect_and_classify(image_path, output_folder):
    # Load the image
    image = cv2.imread(image_path)

    # Detect guns using YOLOv9c
    results = yolo_model(image)

    # Extract bounding boxes for detected guns
    gun_boxes = results[0].boxes

    # Initialize counts for each class
    class_counts = {'Class 1': 0, 'Class 2': 0}

    # For each detected gun, classify it using EfficientNet
    if gun_boxes is not None and len(gun_boxes) > 0:
        for box in gun_boxes:
            # Get bounding box coordinates, confidence, and class
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            conf = box.conf[0].item()
            cls = int(box.cls[0].item())

            # Use names dictionary for class label (if applicable)
            class_label = results[0].names[cls]

            # Crop the gun region from the image
            gun_crop = image[y1:y2, x1:x2]

            # Preprocess for EfficientNet
            gun_crop_preprocessed = preprocess_for_effnet(gun_crop)

            # Classify the gun with EfficientNet
            predictions = effnet_model.predict(gun_crop_preprocessed)
            predicted_class = np.argmax(predictions, axis=1)

            # Update counts based on the predicted class
            if predicted_class == 0:
                class_counts['Class 1'] += 1
            else:
                class_counts['Class 2'] += 1

            # Annotate the image with classification result
            cv2.putText(image, f'{class_label} ({conf:.2f})', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
            cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

    # Save the result to the specified output folder
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    output_filename = os.path.join(output_folder, f"output_{os.path.basename(image_path)}")
    cv2.imwrite(output_filename, image)

    # Print the counts of each class
    # print(f"Total Class 1: {class_counts['Class 1']}")
    # print(f"Total Class 2: {class_counts['Class 2']}")

    return output_filename


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.applications.efficientnet import preprocess_input
import os

# Preprocess image for EfficientNet
def preprocess_for_effnet(crop):
    crop_resized = cv2.resize(crop, (300, 300))  # Resize to 300x300 for EfficientNet B3
    crop_preprocessed = preprocess_input(crop_resized)  # Apply EfficientNet preprocessing
    crop_preprocessed = np.expand_dims(crop_preprocessed, axis=0)  # Add batch dimension
    return crop_preprocessed

# Detection and classification pipeline for a single image
def detect_and_classify(image_path, output_folder):
    # Load the image
    image = cv2.imread(image_path)

    # Detect guns using YOLOv9c
    results = yolo_model(image)
    print(results[0].boxes)
    # gun_boxes = results.xyxy[0]  # Extract bounding boxes for detected guns
    gun_boxes = results.pred[0].cpu().numpy()

    # For each detected gun, classify it using EfficientNet
    for box in gun_boxes:
        x1, y1, x2, y2, conf, cls = map(int, box[:6])
        gun_crop = image[y1:y2, x1:x2]  # Crop the gun region

        # Preprocess for EfficientNet
        gun_crop_preprocessed = preprocess_for_effnet(gun_crop)

        # Classify the gun with EfficientNet
        predictions = effnet_model.predict(gun_crop_preprocessed)
        predicted_class = np.argmax(predictions, axis=1)
        class_label = 'Class 1' if predicted_class == 0 else 'Class 2'

        # Annotate the image with classification result
        cv2.putText(image, f'{class_label}', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
        cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

    # Save the result to the specified output folder
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    output_filename = os.path.join(output_folder, f"output_{os.path.basename(image_path)}")
    cv2.imwrite(output_filename, image)

    return output_filename